In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

2025-01-19 20:46:29.252042: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-19 20:46:29.333729: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-19 20:46:29.409361: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737330389.487190   17050 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737330389.508816   17050 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-19 20:46:29.684909: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:
def get_data():
    #TODO CRIAR A FUNÇÃO QUE BAIXA OS DADOS DO WANDB
    data = pd.read_csv('../results/balanced.csv')
    return data

In [4]:
def vectorize_data(data):
    vectorizer = TfidfVectorizer(max_features=5000)
    X_tfidf = vectorizer.fit_transform(data).toarray()
    return X_tfidf

In [5]:
def create_model():
    model = Sequential([
        Dense(64, activation='relu', input_shape=(5000,)),
        Dense(32, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid') 
    ])

    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    model.summary()
    return model

In [6]:
def compile_model():
    df = get_data()

    X_Tfidf = vectorize_data(df['clean_text'])
    X_train, X_test, y_train, y_test = train_test_split(X_Tfidf, df['label'], test_size=0.2, random_state=42)
    model = create_model()

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

In [7]:
compile_model()

/home/gabs/miniconda3/envs/atr/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-19 20:46:39.295215: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │       320,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 323,233 (1.23 MB)

 Trainable params: 323,233 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


2025-01-19 20:46:40.061553: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 616180000 exceeds 10% of free system memory.


963/963 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9280 - loss: 0.1764 - val_accuracy: 0.9875 - val_loss: 0.0391
Epoch 2/10
963/963 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9942 - loss: 0.0188 - val_accuracy: 0.9873 - val_loss: 0.0393
Epoch 3/10
963/963 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9974 - loss: 0.0088 - val_accuracy: 0.9871 - val_loss: 0.0469
Epoch 4/10
963/963 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9988 - loss: 0.0044 - val_accuracy: 0.9861 - val_loss: 0.0587
Epoch 5/10
963/963 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9989 - loss: 0.0035 - val_accuracy: 0.9865 - val_loss: 0.0596
Epoch 6/10
963/963 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9993 - loss: 0.0024 - val_accuracy: 0.9869 - val_loss: 0.0549
Epoch 7/10
963/963 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9989 - loss: 0.0028 - val_accuracy: 0.9864 - val_loss: 0.0608
Epoch 8/10
963/963 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9992 - loss: 0.0019 - val_accuracy: 0.9869 - val_